In [12]:
import tensorflow as tf
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Load Data

In [13]:
df = pd.read_csv('../data/Luke_hair_loss_documentation.csv')
df.head()

,date,hair_loss,stay_up_late,pressure_level,coffee_consumed,brain_working_duration,school_assesssment,stress_level,shampoo_brand,swimming,hair_washing,hair_grease,dandruff,libido
0,1/01/2021,Few,2,Low,0,1,None,Low,Pantene,No,Y,3.0,None,1
1,2/01/2021,Few,0,Low,0,3,None,Low,Pantene,No,N,1.0,None,1
2,3/01/2021,Medium,3,Low,1,0,None,Low,Pantene,Yes,Y,2.0,None,2
3,4/01/2021,Few,2,Low,0,1,None,Low,Pantene,No,N,3.0,None,3
4,5/01/2021,Few,2,Low,0,1,None,Low,Pantene,No,Y,1.0,None,2


Processing Data

In [14]:
df['dandruff'].replace('None',0,inplace=True)
df['dandruff'].replace('Few',1,inplace=True)
df['dandruff'].replace('Many',2, inplace=True)

df['hair_loss'].replace("Few", 0, inplace=True)
df['hair_loss'].replace("Medium", 0, inplace=True)
df['hair_loss'].replace("Many", 1, inplace=True)
df['hair_loss'].replace("A lot", 1, inplace=True)

df['swimming'].replace('Yes', 1, inplace=True)
df['swimming'].replace('No', 0, inplace=True)

df.replace('Low', 0, inplace=True)
df.replace('Medium', 1, inplace=True)
df.replace('High', 2, inplace=True)
df.replace('Very High', 3, inplace=True)



df.head()

,date,hair_loss,stay_up_late,pressure_level,coffee_consumed,brain_working_duration,school_assesssment,stress_level,shampoo_brand,swimming,hair_washing,hair_grease,dandruff,libido
0,1/01/2021,0,2,0,0,1,None,0,Pantene,0,Y,3.0,0,1
1,2/01/2021,0,0,0,0,3,None,0,Pantene,0,N,1.0,0,1
2,3/01/2021,0,3,0,1,0,None,0,Pantene,1,Y,2.0,0,2
3,4/01/2021,0,2,0,0,1,None,0,Pantene,0,N,3.0,0,3
4,5/01/2021,0,2,0,0,1,None,0,Pantene,0,Y,1.0,0,2


In [15]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    400 non-null    object 
 1   hair_loss               400 non-null    int64  
 2   stay_up_late            400 non-null    int64  
 3   pressure_level          400 non-null    int64  
 4   coffee_consumed         400 non-null    int64  
 5   brain_working_duration  400 non-null    int64  
 6   school_assesssment      400 non-null    object 
 7   stress_level            400 non-null    int64  
 8   shampoo_brand           400 non-null    object 
 9   swimming                400 non-null    int64  
 10  hair_washing            400 non-null    object 
 11  hair_grease             396 non-null    float64
 12  dandruff                400 non-null    int64  
 13  libido                  400 non-null    int64  
dtypes: float64(1), int64(9), object(4)
memory 

In [16]:
plot_cols = df.corr().loc['stay_up_late':, 'hair_loss'].abs().sort_values(ascending=False)
plot_cols

dandruff                  0.808751
stress_level              0.784986
pressure_level            0.779733
hair_grease               0.682490
coffee_consumed           0.671447
brain_working_duration    0.591093
stay_up_late              0.350551
swimming                  0.240211
libido                    0.140884
Name: hair_loss, dtype: float64

In [17]:
X_df= df[['stress_level', 'pressure_level','dandruff']]


In [18]:
y_data = df['hair_loss'].to_numpy().astype(np.float16)
print(y_data.shape)

df.drop('hair_loss', axis=1, inplace=True)

X_data = X_df.to_numpy().astype(np.float16)

print(X_data.shape)

(400,)
(400, 2)


In [19]:
# Train and Test Data Split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.05, random_state=42)
print(X_train.shape)
print(X_test.shape)

(380, 2)
(20, 2)


In [22]:
# Model building
model = tf.keras.Sequential()

model.add(tf.keras.layers.Input(shape=(3)))

model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['acc'], optimizer='adam')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 64)                192       
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 16)                528       
                                                                 
 dense_11 (Dense)            (None, 1)                 17        
                                                                 
Total params: 2,817
Trainable params: 2,817
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Training
model.fit(X_train, y_train,epochs=50,batch_size=64,validation_split=0.10, verbose=2)


Epoch 1/50


/home/inlab/.local/lib/python3.8/site-packages/keras/backend.py:5673: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


6/6 - 0s - loss: 0.6907 - acc: 0.7807 - val_loss: 0.6762 - val_acc: 0.8158 - 399ms/epoch - 67ms/step
Epoch 2/50
6/6 - 0s - loss: 0.6752 - acc: 0.8158 - val_loss: 0.6580 - val_acc: 0.8947 - 18ms/epoch - 3ms/step
Epoch 3/50
6/6 - 0s - loss: 0.6620 - acc: 0.8947 - val_loss: 0.6446 - val_acc: 0.8947 - 17ms/epoch - 3ms/step
Epoch 4/50
6/6 - 0s - loss: 0.6497 - acc: 0.8947 - val_loss: 0.6309 - val_acc: 0.8947 - 16ms/epoch - 3ms/step
Epoch 5/50
6/6 - 0s - loss: 0.6359 - acc: 0.8947 - val_loss: 0.6162 - val_acc: 0.8947 - 17ms/epoch - 3ms/step
Epoch 6/50
6/6 - 0s - loss: 0.6210 - acc: 0.8947 - val_loss: 0.5970 - val_acc: 0.8947 - 20ms/epoch - 3ms/step
Epoch 7/50
6/6 - 0s - loss: 0.6022 - acc: 0.8947 - val_loss: 0.5748 - val_acc: 0.8947 - 24ms/epoch - 4ms/step
Epoch 8/50
6/6 - 0s - loss: 0.5792 - acc: 0.8947 - val_loss: 0.5510 - val_acc: 0.8947 - 17ms/epoch - 3ms/step
Epoch 9/50
6/6 - 0s - loss: 0.5531 - acc: 0.8947 - val_loss: 0.5209 - val_acc: 0.8947 - 16ms/epoch - 3ms/step
Epoch 10/50
6/6 - 0

In [ ]:
# Evaluating
model.evaluate(X_test, y_test)


In [ ]:
model.predict(np.array([[2,2,2]]))


In [ ]:
# Serialize the model and save
import joblib
joblib.dump(model, 'hairloss.pkl')
print("Hair Loss Model Saved")

# Load the model
model = joblib.load('hairloss.pkl')

# Save features from training
model_columns = list(X_df.columns)
joblib.dump(model_columns, 'hairloss_columns.pkl')
print("Hair loss Model Columns Saved")